In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from Modules2.preprocessor import Preprocessor
from Modules2.setup_logger import setup_logger
import os
import pymongo
import json


class MongoDB:
    """
    Class for inserting data from csv file to mongodb database
    Parameters:
    -----------
    dbname: name of database
    collection: name of collection
    """
    def __init__(self,dbname,collection):
        self.client = pymongo.MongoClient('mongodb://localhost:27017/')
        self.dbname = self.client[dbname]
        self.collection = self.dbname[collection]      

        self.preprocess = Preprocessor()  
        self.log = setup_logger("database_log", "logs2/mongodb_database.log")
        self.log.info("MongoDB database Created.....")
        

    def insert_data(self):
        """
        Method: insert_data
        Function: insert data from csv file to mongodb database
        input: None
        output: csv file
        on_error: log error

        version 1.0
        """
        try:
            self.log.info("Insertion table into MongoDB database... Started")
            df=self.preprocess.drop_data()
            records= list(json.loads(df.T.to_json()).values())
            self.collection.insert_many(records)
            self.log.info("All the records are inserted into MongoDB database. Task completed......")
            
            self.save_to_csv()
            self.log.info("Records from MongoDB database successfully saved into csv file......")
            
        except Exception as e:
            self.log.error("Error in insert train data in mongoDB server " + str(e))
            
    def save_to_csv(self):
        """
        Method: save_to_csv
        Function: save data from mongodb database to csv file
        input: None
        output: csv file
        on_error: log error

        version 1.0
        """
        try:
            self.log.info("Datafrom MongoDB database is getting started to save into csv file..")
            
            result=self.collection.find()
            df = pd.DataFrame(result).drop(columns = '_id')
                
            if not os.path.isdir('./Data_from_database/'):
                os.mkdir('./Data_from_database/')
            df.to_csv('./Data_from_database/' + 'Preprocessed_data.csv',index=False)
            
            return df
        except Exception as e:
            self.log.error("Error occured while dropping columns from data " + str(e))